This code is meant to train and evaluate a custom Inception CNN model toward detecting pneumonia and differentiating bacterial and viral pneumonia in pediatric chest radiographs. You can use your own custom data to train and validate these models.

In [ ]:
#load libraries
from __future__ import print_function

import numpy as np
from keras.utils import plot_model
from keras.models import model_from_json
from keras.models import Sequential
from keras.models import Model
from keras import layers
from keras.layers import Conv2D, Dense, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, Concatenate
from keras.layers import Input
import time
from keras.optimizers import SGD
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from evaluation import plot_confusion_matrix
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils import plot_model
%matplotlib inline

In [ ]:
# Loading the data
train_data_dir = 'cxr_normal_pneumonia_1024/train'
test_data_dir = 'cxr_normal_pneumonia_1024/test'
img_width = 1024
img_height = 1024
channel = 3
input_img = Input(shape = (img_width, img_height, channel))
epochs = 100
batch_size = 8 #vary this parameter depending on your GPU capacity
num_classes= 2 #[pneumonia, normal] [bacterial, viral]

In [ ]:
#Load the InceptionV3 based model layers.
tower_1a = Conv2D(4, (1,1), padding='same', activation='relu', strides=2)(input_img)
tower_2a = Conv2D(4, (1,1), padding='same', activation='relu')(input_img)
tower_2a = Conv2D(4, (3,3), padding='same', activation='relu', strides=2)(tower_2a)
tower_3a = AveragePooling2D((3,3), strides=(2,2), padding='same')(input_img)
tower_3a = Conv2D(4, (3,3), padding='same', activation='relu')(tower_3a)
tower_4a = Conv2D(4, (1,1), padding='same', activation='relu')(input_img)
tower_4a = Conv2D(4, (3,3), padding='same', activation='relu')(tower_4a)
tower_4a = Conv2D(4, (3,3), padding='same', activation='relu', strides=2)(tower_4a)
output1 = layers.concatenate([tower_1a, tower_2a, tower_3a, tower_4a], axis = -1)

#module2
tower_1b = Conv2D(8, (1,1), padding='same', activation='relu', strides=2)(output1)
tower_2b = Conv2D(8, (1,1), padding='same', activation='relu')(output1)
tower_2b = Conv2D(8, (3,3), padding='same', activation='relu', strides=2)(tower_2b)
tower_3b = AveragePooling2D((3,3), strides=(2,2), padding='same')(output1)
tower_3b = Conv2D(8, (3,3), padding='same', activation='relu')(tower_3b)
tower_4b = Conv2D(8, (1,1), padding='same', activation='relu')(output1)
tower_4b = Conv2D(8, (3,3), padding='same', activation='relu')(tower_4b)
tower_4b = Conv2D(8, (3,3), padding='same', activation='relu', strides=2)(tower_4b)
output2 = layers.concatenate([tower_1b, tower_2b, tower_3b, tower_4b], axis = -1)

#module3
tower_1c = Conv2D(16, (1,1), padding='same', activation='relu', strides=2)(output2)
tower_2c = Conv2D(16, (1,1), padding='same', activation='relu')(output2)
tower_2c = Conv2D(16, (3,3), padding='same', activation='relu', strides=2)(tower_2c)
tower_3c = AveragePooling2D((3,3), strides=(2,2), padding='same')(output2)
tower_3c = Conv2D(16, (3,3), padding='same', activation='relu')(tower_3c)
tower_4c = Conv2D(16, (1,1), padding='same', activation='relu')(output2)
tower_4c = Conv2D(16, (3,3), padding='same', activation='relu')(tower_4c)
tower_4c = Conv2D(16, (3,3), padding='same', activation='relu', strides=2)(tower_4c)
output3 = layers.concatenate([tower_1c, tower_2c, tower_3c, tower_4c], axis = -1)

#module4
tower_1d = Conv2D(32, (1,1), padding='same', activation='relu', strides=2)(output3)
tower_2d = Conv2D(32, (1,1), padding='same', activation='relu')(output3)
tower_2d = Conv2D(32, (3,3), padding='same', activation='relu', strides=2)(tower_2d)
tower_3d = AveragePooling2D((3,3), strides=(2,2), padding='same')(output3)
tower_3d = Conv2D(32, (3,3), padding='same', activation='relu')(tower_3d)
tower_4d = Conv2D(32, (1,1), padding='same', activation='relu')(output3)
tower_4d = Conv2D(32, (3,3), padding='same', activation='relu')(tower_4d)
tower_4d = Conv2D(32, (3,3), padding='same', activation='relu', strides=2)(tower_4d)
output4 = layers.concatenate([tower_1d, tower_2d, tower_3d, tower_4d], axis = -1)

#module5
tower_1e = Conv2D(64, (1,1), padding='same', activation='relu', strides=2)(output4)
tower_2e = Conv2D(64, (1,1), padding='same', activation='relu')(output4)
tower_2e = Conv2D(64, (3,3), padding='same', activation='relu', strides=2)(tower_2e)
tower_3e = AveragePooling2D((3,3), strides=(2,2), padding='same')(output4)
tower_3e = Conv2D(64, (3,3), padding='same', activation='relu')(tower_3e)
tower_4e = Conv2D(64, (1,1), padding='same', activation='relu')(output4)
tower_4e = Conv2D(64, (3,3), padding='same', activation='relu')(tower_4e)
tower_4e = Conv2D(64, (3,3), padding='same', activation='relu', strides=2)(tower_4e)
output5 = layers.concatenate([tower_1e, tower_2e, tower_3e, tower_4e], axis = -1)

#module6
tower_1f = Conv2D(128, (1,1), padding='same', activation='relu', strides=2)(output5)
tower_2f = Conv2D(128, (1,1), padding='same', activation='relu')(output5)
tower_2f = Conv2D(128, (3,3), padding='same', activation='relu', strides=2)(tower_2f)
tower_3f = AveragePooling2D((3,3), strides=(2,2), padding='same')(output5)
tower_3f = Conv2D(128, (3,3), padding='same', activation='relu')(tower_3f)
tower_4f = Conv2D(128, (1,1), padding='same', activation='relu')(output5)
tower_4f = Conv2D(128, (3,3), padding='same', activation='relu')(tower_4f)
tower_4f = Conv2D(128, (3,3), padding='same', activation='relu', strides=2)(tower_4f)

#concatenating the outputs
output6 = layers.concatenate([tower_1f, tower_2f, tower_3f, tower_4f], axis = -1)

#adding global average pooling layer
output = GlobalAveragePooling2D()(output6)

#adding the final dense layer
out = Dense(num_classes, activation='softmax', name='predictions')(output)

#creating the model
model = Model(inputs = input_img, outputs = out)

#generating model summary
model.summary()

#plot the model
plot_model(model, to_file='custom_inceptionv3_model.png')

We performed five-fold cross validation in this study. We have however shown running the script with a sample data fold. We performed no augmentation other than rescaling.We allocated 20% of the training data at random for validation.

In [ ]:
#declaring image data generators
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2) #taking 20% of training for validation 

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(1024, 1024),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(1024, 1024),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(1024, 1024),
        batch_size=batch_size,
        class_mode='categorical',shuffle=False)

#count the number of samples
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

#check the class indices
train_generator.class_indices
validation_generator.class_indices
test_generator.class_indices

Allocate balanced weights to penalize over-represented classes

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

In [ ]:
#fix the optimizer
sgd = SGD(lr=0.01, decay=1e-4, momentum=0.95, nesterov=True) #optimize to your requirements

#compile the model
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#compute the training time
start = time.time()

#give the path to store the model weights
filepath = 'weights/' + model.name + '.{epoch:02d}-{val_acc:.4f}.hdf5'

#save only the best weights
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_weights_only=True, save_best_only=True, mode='max', period=1)

#visualize model performance using TensorBoard
tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=batch_size)
callbacks_list = [checkpoint, tensor_board]

#train model
custom_inception_history = model.fit_generator(
      train_generator,
      steps_per_epoch=nb_train_samples // batch_size, 
      epochs=epochs,
      validation_data=validation_generator,
      callbacks=callbacks_list,
      class_weight = class_weights,
      validation_steps=nb_validation_samples // batch_size, 
      verbose=1)

#print the total time taken for training
print(time.time()-start)

In [ ]:
#Testing the model's performance by loading the model weights

model.load_weights('weights/model_1.01-0.7423.hdf5') #change this to your path and model weights
custom_inception_y_pred = model.predict_generator(test_generator, nb_test_samples/batch_size, workers=1)

#true labels
Y_test=test_generator.classes

#print the shape of y_pred and Y_test
print(custom_inception_y_pred.shape)
print(Y_test.shape)

#measure accuracy
custom_inception_model_accuracy=accuracy_score(Y_test,custom_inception_y_pred.argmax(axis=-1))
print('The accuracy of custom Inception model is: ', custom_inception_model_accuracy)

In [ ]:
#print classification report
target_names = ['class 0(normal)', 'class 1(pneumonia)'] #from the generator.class_indices
print(classification_report(Y_test,custom_inception_y_pred.argmax(axis=-1),
                                                              target_names=target_names, digits=4))

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,custom_inception_y_pred.argmax(axis=-1))
np.set_printoptions(precision=4)
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for custom Inception model without normalization')
plt.show()

In [ ]:
#store the predictions in .csv files
print(custom_inception_y_pred.argmax(axis=-1))
print(Y_test)

np.savetxt('custom_Inception_y_pred.csv',custom_inception_y_pred.argmax(axis=-1),fmt='%i',delimiter = ",")
np.savetxt('Y_test.csv',Y_test,fmt='%i',delimiter = ",")

In [ ]:
#compute the ROC-AUC values
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test, custom_inception_y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
fig=plt.figure(figsize=(15,10), dpi=100)
ax = fig.add_subplot(1, 1, 1)
major_ticks = np.arange(0.0, 1.0, 0.10)
minor_ticks = np.arange(0.0, 1.0, 0.10)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(fpr[1], tpr[1], color='red',
         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics')
plt.legend(loc="lower right")
plt.show()